In [2]:
## Import Packages
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import scipy.io as scio
import os
import pickle
import Area_Avg
from netCDF4 import Dataset
import warnings
warnings.filterwarnings('ignore')

In [3]:
LLL = Dataset('CESM-WACCM Ocean Grid Cell Thickness/thkcello_CESM1-WACCM_interp.nc')
WACCM_Lat = np.squeeze(LLL.variables['lat'])
WACCM_Arctic_Lat = WACCM_Lat[150:]
WACCM_Lon = np.squeeze(LLL.variables['lon'])

In [4]:
RCP45_wm_TmSrs = np.zeros([5,732,180,360])
Sims = ['1','2','3','4','5']
Years = np.arange(60)

for i in range(len(Sims)):
    RCP45_wm_TmSrs[i,:,:,:] = pickle.load(open\
                    ('Time Series Data/RCP45_OHC_TimeSeries_Wm_EM'+Sims[i]+'.pickle',\
                    'rb'),encoding='latin1')

In [5]:
#stack monthly values so we have two separate time dimensions, one for years, one for months
RCP45_wm_TmSrs_stacked = np.zeros([5,60,12,180,360])

start = 12
finish = 24

for i in range(60):
    for j in range(5):
        RCP45_wm_TmSrs_stacked[j,i,:,:,:] = np.stack(\
        RCP45_wm_TmSrs[j,start:finish,:,:],axis=0)
        
    start+=12
    finish+=12 

In [6]:
RCP45_wm_TmSrs_Annual = np.mean(RCP45_wm_TmSrs_stacked[:,:,:,:,:],axis=2)
RCP45_wm_TmSrs_MAM = np.mean(RCP45_wm_TmSrs_stacked[:,:,2:5,:,:],axis=2)
RCP45_wm_TmSrs_JJA = np.mean(RCP45_wm_TmSrs_stacked[:,:,5:8,:,:],axis=2)
RCP45_wm_TmSrs_SON = np.mean(RCP45_wm_TmSrs_stacked[:,:,8:11,:,:],axis=2)
RCP45_wm_TmSrs_DJF = np.mean((RCP45_wm_TmSrs_stacked[:,:,0,:,:],\
                             RCP45_wm_TmSrs_stacked[:,:,1,:,:],\
                             RCP45_wm_TmSrs_stacked[:,:,11,:,:]),axis=0)

In [ ]:
RCP45_wm_TmSrs_Annual_AA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_MAM_AA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_JJA_AA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_SON_AA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_DJF_AA = np.zeros([len(Sims),Years.size])

RCP45_wm_TmSrs_Annual_GA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_MAM_GA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_JJA_GA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_SON_GA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_DJF_GA = np.zeros([len(Sims),Years.size])

for j in range(len(Sims)):
    RCP45_wm_TmSrs_Annual_AA[j,:] = Area_Avg.LatLonavg_Time(\
        RCP45_wm_TmSrs_Annual[j,:,150:,:],WACCM_Arctic_Lat,WACCM_Lon)
    
    RCP45_wm_TmSrs_MAM_AA[j,:] = Area_Avg.LatLonavg_Time(\
        RCP45_wm_TmSrs_MAM[j,:,150:,:],WACCM_Arctic_Lat,WACCM_Lon)
    
    RCP45_wm_TmSrs_JJA_AA[j,:] = Area_Avg.LatLonavg_Time(\
        RCP45_wm_TmSrs_JJA[j,:,150:,:],WACCM_Arctic_Lat,WACCM_Lon)
    
    RCP45_wm_TmSrs_SON_AA[j,:] = Area_Avg.LatLonavg_Time(\
        RCP45_wm_TmSrs_SON[j,:,150:,:],WACCM_Arctic_Lat,WACCM_Lon)
    
    RCP45_wm_TmSrs_DJF_AA[j,:] = Area_Avg.LatLonavg_Time(\
        RCP45_wm_TmSrs_DJF[j,:,150:,:],WACCM_Arctic_Lat,WACCM_Lon)
    
    
    RCP45_wm_TmSrs_Annual_GA[j,:] = Area_Avg.LatLonavg_Time(\
        RCP45_wm_TmSrs_Annual[j,:,:,:],WACCM_Lat,WACCM_Lon)
    
    RCP45_wm_TmSrs_MAM_GA[j,:] = Area_Avg.LatLonavg_Time(\
        RCP45_wm_TmSrs_MAM[j,:,:,:],WACCM_Lat,WACCM_Lon)
    
    RCP45_wm_TmSrs_JJA_GA[j,:] = Area_Avg.LatLonavg_Time(\
        RCP45_wm_TmSrs_JJA[j,:,:,:],WACCM_Lat,WACCM_Lon)
    
    RCP45_wm_TmSrs_SON_GA[j,:] = Area_Avg.LatLonavg_Time(\
        RCP45_wm_TmSrs_SON[j,:,:,:],WACCM_Lat,WACCM_Lon)
    
    RCP45_wm_TmSrs_DJF_GA[j,:] = Area_Avg.LatLonavg_Time(\
        RCP45_wm_TmSrs_DJF[j,:,:,:],WACCM_Lat,WACCM_Lon)

Ocn_GridCell_Area_file = Dataset(\
    'CESM-WACCM Ocean Grid Cell Thickness/areacello_CESM1-WACCM_interp.nc')
Ocn_GridCell_Area = np.squeeze(Ocn_GridCell_Area_file.variables['areacello'])
Ocn_GridCell_Area[Ocn_GridCell_Area == 1e+20] = 0

Ocean_Area_Arctic = np.sum(Ocn_GridCell_Area[150:,:])
Ocean_Area_Global = np.sum(Ocn_GridCell_Area)

Weights_Arctic = Ocn_GridCell_Area[150:,:]/Ocean_Area_Arctic
Weights_Global = Ocn_GridCell_Area/Ocean_Area_Global

RCP45_wm_TmSrs_Annual_AA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_MAM_AA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_JJA_AA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_SON_AA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_DJF_AA = np.zeros([len(Sims),Years.size])

RCP45_wm_TmSrs_Annual_GA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_MAM_GA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_JJA_GA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_SON_GA = np.zeros([len(Sims),Years.size])
RCP45_wm_TmSrs_DJF_GA = np.zeros([len(Sims),Years.size])

for i in range(5):
    for j in range(60):
        RCP45_wm_TmSrs_Annual_AA[i,j] = \
        np.sum(RCP45_wm_TmSrs_Annual[i,j,150:,:]*Weights_Arctic,axis=(0,1))
        
        RCP45_wm_TmSrs_MAM_AA[i,j] = \
        np.sum(RCP45_wm_TmSrs_MAM[i,j,150:,:]*Weights_Arctic,axis=(0,1))
        
        RCP45_wm_TmSrs_JJA_AA[i,j] = \
        np.sum(RCP45_wm_TmSrs_JJA[i,j,150:,:]*Weights_Arctic,axis=(0,1))
        
        RCP45_wm_TmSrs_SON_AA[i,j] = \
        np.sum(RCP45_wm_TmSrs_SON[i,j,150:,:]*Weights_Arctic,axis=(0,1))
        
        RCP45_wm_TmSrs_DJF_AA[i,j] = \
        np.sum(RCP45_wm_TmSrs_DJF[i,j,150:,:]*Weights_Arctic,axis=(0,1))
        
        
        RCP45_wm_TmSrs_Annual_GA[i,j] = \
        np.sum(RCP45_wm_TmSrs_Annual[i,j,:,:]*Weights_Global,axis=(0,1))
        
        RCP45_wm_TmSrs_MAM_GA[i,j] = \
        np.sum(RCP45_wm_TmSrs_MAM[i,j,:,:]*Weights_Global,axis=(0,1))
        
        RCP45_wm_TmSrs_JJA_GA[i,j] = \
        np.sum(RCP45_wm_TmSrs_JJA[i,j,:,:]*Weights_Global,axis=(0,1))
        
        RCP45_wm_TmSrs_SON_GA[i,j] = \
        np.sum(RCP45_wm_TmSrs_SON[i,j,:,:]*Weights_Global,axis=(0,1))
        
        RCP45_wm_TmSrs_DJF_GA[i,j] = \
        np.sum(RCP45_wm_TmSrs_DJF[i,j,:,:]*Weights_Global,axis=(0,1))

In [ ]:
RCP45_dOHC_Annual_AA = np.zeros([5])
RCP45_dOHC_MAM_AA = np.zeros([5])
RCP45_dOHC_JJA_AA = np.zeros([5])
RCP45_dOHC_SON_AA = np.zeros([5])
RCP45_dOHC_DJF_AA = np.zeros([5])

RCP45_dOHC_Annual_GA = np.zeros([5])
RCP45_dOHC_MAM_GA = np.zeros([5])
RCP45_dOHC_JJA_GA = np.zeros([5])
RCP45_dOHC_SON_GA = np.zeros([5])
RCP45_dOHC_DJF_GA = np.zeros([5])

for i in range(5):
    RCP45_dOHC_Annual_AA[i] = (np.mean(RCP45_wm_TmSrs_Annual_AA[i,50:]))-\
                                     (np.mean(RCP45_wm_TmSrs_Annual_AA[i,:10]))
    RCP45_dOHC_MAM_AA[i] = (np.mean(RCP45_wm_TmSrs_MAM_AA[i,50:]))-\
                                     (np.mean(RCP45_wm_TmSrs_MAM_AA[i,:10]))
    RCP45_dOHC_JJA_AA[i] = (np.mean(RCP45_wm_TmSrs_JJA_AA[i,50:]))-\
                                     (np.mean(RCP45_wm_TmSrs_JJA_AA[i,:10]))
    RCP45_dOHC_SON_AA[i] = (np.mean(RCP45_wm_TmSrs_SON_AA[i,50:]))-\
                                     (np.mean(RCP45_wm_TmSrs_SON_AA[i,:10]))
    RCP45_dOHC_DJF_AA[i] = (np.mean(RCP45_wm_TmSrs_DJF_AA[i,50:]))-\
                                     (np.mean(RCP45_wm_TmSrs_DJF_AA[i,:10]))

    RCP45_dOHC_Annual_GA[i] = (np.mean(RCP45_wm_TmSrs_Annual_GA[i,50:]))-\
                                     (np.mean(RCP45_wm_TmSrs_Annual_GA[i,:10]))
    RCP45_dOHC_MAM_GA[i] = (np.mean(RCP45_wm_TmSrs_MAM_GA[i,50:]))-\
                                     (np.mean(RCP45_wm_TmSrs_MAM_GA[i,:10]))
    RCP45_dOHC_JJA_GA[i] = (np.mean(RCP45_wm_TmSrs_JJA_GA[i,50:]))-\
                                     (np.mean(RCP45_wm_TmSrs_JJA_GA[i,:10]))
    RCP45_dOHC_SON_GA[i] = (np.mean(RCP45_wm_TmSrs_SON_GA[i,50:]))-\
                                     (np.mean(RCP45_wm_TmSrs_SON_GA[i,:10]))
    RCP45_dOHC_DJF_GA[i] = (np.mean(RCP45_wm_TmSrs_DJF_GA[i,50:]))-\
                                     (np.mean(RCP45_wm_TmSrs_DJF_GA[i,:10]))

In [18]:
print('Annual :',RCP45_dOHC_Annual_AA)
print('\n')
print('MAM :',RCP45_dOHC_MAM_AA)
print('JJA :',RCP45_dOHC_JJA_AA)
print('SON :',RCP45_dOHC_SON_AA)
print('DJF :',RCP45_dOHC_DJF_AA)

Annual : [ 0.62041928 -0.05726521  0.95088589  0.87735605  0.24416392]


MAM : [1.84363109 0.28626861 1.68911958 1.30086973 1.45289266]
JJA : [5.0014305  2.75442306 8.0859801  5.67689806 5.83927317]
SON : [-2.38287853 -0.01253614 -3.08202006 -1.1381071  -2.25310078]
DJF : [-1.98050594 -3.25721635 -2.88953608 -2.3302365  -4.06240937]


In [17]:
print('Annual :',RCP45_dOHC_Annual_GA)
print('\n')
print('MAM :',RCP45_dOHC_MAM_GA)
print('JJA :',RCP45_dOHC_JJA_GA)
print('SON :',RCP45_dOHC_SON_GA)
print('DJF :',RCP45_dOHC_DJF_GA)

Annual : [ 0.23420564 -0.12349275  0.08701003  0.06231212  0.42652633]


MAM : [0.45405305 0.33632228 0.34978655 0.80877174 0.76308329]
JJA : [-0.35675157 -0.55487909  0.38917308  0.25423325  0.20115741]
SON : [ 0.72466735 -0.06079519 -0.05361085  0.1767871   0.7524815 ]
DJF : [ 0.11485372 -0.21461899 -0.33730867 -0.9905436  -0.01061689]


In [46]:
RCP45_OCN_EnergyB_Annual_AA_file = open('RCP45_OCN_EnergyB_AA.pickle','wb')
pickle.dump(RCP45_dOHC_Annual_AA,RCP45_OCN_EnergyB_Annual_AA_file)
RCP45_OCN_EnergyB_Annual_AA_file.close()

#seasonal
RCP45_OCN_EnergyB_MAM_AA_file = open('RCP45_OCN_EnergyB_MAM_AA.pickle','wb')
pickle.dump(RCP45_dOHC_MAM_AA,RCP45_OCN_EnergyB_MAM_AA_file)
RCP45_OCN_EnergyB_MAM_AA_file.close()

RCP45_OCN_EnergyB_JJA_AA_file = open('RCP45_OCN_EnergyB_JJA_AA.pickle','wb')
pickle.dump(RCP45_dOHC_JJA_AA,RCP45_OCN_EnergyB_JJA_AA_file)
RCP45_OCN_EnergyB_JJA_AA_file.close()

RCP45_OCN_EnergyB_SON_AA_file = open('RCP45_OCN_EnergyB_SON_AA.pickle','wb')
pickle.dump(RCP45_dOHC_SON_AA,RCP45_OCN_EnergyB_SON_AA_file)
RCP45_OCN_EnergyB_SON_AA_file.close()

RCP45_OCN_EnergyB_DJF_AA_file = open('RCP45_OCN_EnergyB_DJF_AA.pickle','wb')
pickle.dump(RCP45_dOHC_DJF_AA,RCP45_OCN_EnergyB_DJF_AA_file)
RCP45_OCN_EnergyB_DJF_AA_file.close()


RCP45_OCN_EnergyB_Annual_GA_file = open('RCP45_OCN_EnergyB_GA.pickle','wb')
pickle.dump(RCP45_dOHC_Annual_GA,RCP45_OCN_EnergyB_Annual_GA_file)
RCP45_OCN_EnergyB_Annual_GA_file.close()

#seasonal
RCP45_OCN_EnergyB_MAM_GA_file = open('RCP45_OCN_EnergyB_MAM_GA.pickle','wb')
pickle.dump(RCP45_dOHC_MAM_GA,RCP45_OCN_EnergyB_MAM_GA_file)
RCP45_OCN_EnergyB_MAM_GA_file.close()

RCP45_OCN_EnergyB_JJA_GA_file = open('RCP45_OCN_EnergyB_JJA_GA.pickle','wb')
pickle.dump(RCP45_dOHC_JJA_GA,RCP45_OCN_EnergyB_JJA_GA_file)
RCP45_OCN_EnergyB_JJA_GA_file.close()

RCP45_OCN_EnergyB_SON_GA_file = open('RCP45_OCN_EnergyB_SON_GA.pickle','wb')
pickle.dump(RCP45_dOHC_SON_GA,RCP45_OCN_EnergyB_SON_GA_file)
RCP45_OCN_EnergyB_SON_GA_file.close()

RCP45_OCN_EnergyB_DJF_GA_file = open('RCP45_OCN_EnergyB_DJF_GA.pickle','wb')
pickle.dump(RCP45_dOHC_DJF_GA,RCP45_OCN_EnergyB_DJF_GA_file)
RCP45_OCN_EnergyB_DJF_GA_file.close()

In [47]:
#feedback now
LLL3 = Dataset('CAM3 Kernels/CAM3_Albedo_Kernel.nc')
CAM3_Lat = np.squeeze(LLL3.variables['lat'])
CAM3_Lon = np.squeeze(LLL3.variables['lon'])

RCP45_dST_CAM3Grid = np.swapaxes(pickle.load(open(\
   "Future Projection CC Responses/RCP45_dST_CAM3Grid.pickle","rb"),encoding='latin1'),2,3)

RCP45_dST_CAM3Grid_AA = np.zeros([5,12])
RCP45_dST_CAM3Grid_GA = np.zeros([5,12])

for i in range(5):
    RCP45_dST_CAM3Grid_AA[i,:] = Area_Avg.LatLonavg_Time(\
    RCP45_dST_CAM3Grid[i,:,54:,:],CAM3_Lat[54:],CAM3_Lon)
    
    RCP45_dST_CAM3Grid_GA[i,:] = Area_Avg.LatLonavg_Time(\
    RCP45_dST_CAM3Grid[i,:,:,:],CAM3_Lat,CAM3_Lon)

RCP45_dST_Annual_CAM3Grid_AA = np.mean(RCP45_dST_CAM3Grid_AA,axis=1)
RCP45_dST_SON_CAM3Grid_AA = np.mean(RCP45_dST_CAM3Grid_AA[:,8:11],axis=1)
RCP45_dST_JJA_CAM3Grid_AA = np.mean(RCP45_dST_CAM3Grid_AA[:,5:8],axis=1)
RCP45_dST_MAM_CAM3Grid_AA = np.mean(RCP45_dST_CAM3Grid_AA[:,2:5],axis=1)
RCP45_dST_DJF_CAM3Grid_AA = np.mean((RCP45_dST_CAM3Grid_AA[:,0],\
                                    RCP45_dST_CAM3Grid_AA[:,1],\
                                    RCP45_dST_CAM3Grid_AA[:,11]),axis=0)

RCP45_dST_Annual_CAM3Grid_GA = np.mean(RCP45_dST_CAM3Grid_GA,axis=1)
RCP45_dST_SON_CAM3Grid_GA = np.mean(RCP45_dST_CAM3Grid_GA[:,8:11],axis=1)
RCP45_dST_JJA_CAM3Grid_GA = np.mean(RCP45_dST_CAM3Grid_GA[:,5:8],axis=1)
RCP45_dST_MAM_CAM3Grid_GA = np.mean(RCP45_dST_CAM3Grid_GA[:,2:5],axis=1)
RCP45_dST_DJF_CAM3Grid_GA = np.mean((RCP45_dST_CAM3Grid_GA[:,0],\
                                    RCP45_dST_CAM3Grid_GA[:,1],\
                                    RCP45_dST_CAM3Grid_GA[:,11]),axis=0)

In [48]:
RCP45_dOHC_Feedback_Annual_AA = RCP45_dOHC_Annual_AA/RCP45_dST_Annual_CAM3Grid_AA
RCP45_dOHC_Feedback_SON_AA = RCP45_dOHC_SON_AA/RCP45_dST_SON_CAM3Grid_AA
RCP45_dOHC_Feedback_JJA_AA = RCP45_dOHC_JJA_AA/RCP45_dST_JJA_CAM3Grid_AA
RCP45_dOHC_Feedback_MAM_AA = RCP45_dOHC_MAM_AA/RCP45_dST_MAM_CAM3Grid_AA
RCP45_dOHC_Feedback_DJF_AA = RCP45_dOHC_DJF_AA/RCP45_dST_DJF_CAM3Grid_AA

RCP45_dOHC_Feedback_Annual_GA = RCP45_dOHC_Annual_GA/RCP45_dST_Annual_CAM3Grid_GA
RCP45_dOHC_Feedback_SON_GA = RCP45_dOHC_SON_GA/RCP45_dST_SON_CAM3Grid_GA
RCP45_dOHC_Feedback_JJA_GA = RCP45_dOHC_JJA_GA/RCP45_dST_JJA_CAM3Grid_GA
RCP45_dOHC_Feedback_MAM_GA = RCP45_dOHC_MAM_GA/RCP45_dST_MAM_CAM3Grid_GA
RCP45_dOHC_Feedback_DJF_GA = RCP45_dOHC_DJF_GA/RCP45_dST_DJF_CAM3Grid_GA

In [49]:
RCP45_OCN_Feedback_Annual_GA_file = open('RCP45_OCN_Feedback_GA.pickle','wb')
pickle.dump(RCP45_dOHC_Feedback_Annual_GA,RCP45_OCN_Feedback_Annual_GA_file)
RCP45_OCN_Feedback_Annual_GA_file.close()

#seasonal
RCP45_OCN_Feedback_MAM_GA_file = open('RCP45_OCN_Feedback_MAM_GA.pickle','wb')
pickle.dump(RCP45_dOHC_Feedback_MAM_GA,RCP45_OCN_Feedback_MAM_GA_file)
RCP45_OCN_Feedback_MAM_GA_file.close()

RCP45_OCN_Feedback_JJA_GA_file = open('RCP45_OCN_Feedback_JJA_GA.pickle','wb')
pickle.dump(RCP45_dOHC_Feedback_JJA_GA,RCP45_OCN_Feedback_JJA_GA_file)
RCP45_OCN_Feedback_JJA_GA_file.close()

RCP45_OCN_Feedback_SON_GA_file = open('RCP45_OCN_Feedback_SON_GA.pickle','wb')
pickle.dump(RCP45_dOHC_Feedback_SON_GA,RCP45_OCN_Feedback_SON_GA_file)
RCP45_OCN_Feedback_SON_GA_file.close()

RCP45_OCN_Feedback_DJF_GA_file = open('RCP45_OCN_Feedback_DJF_GA.pickle','wb')
pickle.dump(RCP45_dOHC_Feedback_DJF_GA,RCP45_OCN_Feedback_DJF_GA_file)
RCP45_OCN_Feedback_DJF_GA_file.close()


RCP45_OCN_Feedback_Annual_AA_file = open('RCP45_OCN_Feedback_AA.pickle','wb')
pickle.dump(RCP45_dOHC_Feedback_Annual_AA,RCP45_OCN_Feedback_Annual_AA_file)
RCP45_OCN_Feedback_Annual_AA_file.close()

#seasonal
RCP45_OCN_Feedback_MAM_AA_file = open('RCP45_OCN_Feedback_MAM_AA.pickle','wb')
pickle.dump(RCP45_dOHC_Feedback_MAM_AA,RCP45_OCN_Feedback_MAM_AA_file)
RCP45_OCN_Feedback_MAM_AA_file.close()

RCP45_OCN_Feedback_JJA_AA_file = open('RCP45_OCN_Feedback_JJA_AA.pickle','wb')
pickle.dump(RCP45_dOHC_Feedback_JJA_AA,RCP45_OCN_Feedback_JJA_AA_file)
RCP45_OCN_Feedback_JJA_AA_file.close()

RCP45_OCN_Feedback_SON_AA_file = open('RCP45_OCN_Feedback_SON_AA.pickle','wb')
pickle.dump(RCP45_dOHC_Feedback_SON_AA,RCP45_OCN_Feedback_SON_AA_file)
RCP45_OCN_Feedback_SON_AA_file.close()

RCP45_OCN_Feedback_DJF_AA_file = open('RCP45_OCN_Feedback_DJF_AA.pickle','wb')
pickle.dump(RCP45_dOHC_Feedback_DJF_AA,RCP45_OCN_Feedback_DJF_AA_file)
RCP45_OCN_Feedback_DJF_AA_file.close()

## WAVD Now

In [50]:
WAVD_wm_TmSrs_1_3 = np.zeros([3,732,180,360])
WAVD_wm_TmSrs_4_5 = np.zeros([2,720,180,360])
Sims = ['1','2','3']
Sims2 = ['4','5']

for i in range(len(Sims)):
    WAVD_wm_TmSrs_1_3[i,:,:,:] = pickle.load(open\
                    ('Time Series Data/WAVD_OHC_TimeSeries_Wm_EM'+Sims[i]+'.pickle',\
                    'rb'),encoding='latin1')
for i in range(len(Sims2)):
    WAVD_wm_TmSrs_4_5[i,:,:,:] = pickle.load(open\
                    ('Time Series Data/WAVD_OHC_TimeSeries_Wm_EM'+Sims2[i]+'.pickle',\
                    'rb'),encoding='latin1')
    
WAVD_wm_TmSrs_link1 = WAVD_wm_TmSrs_1_3[:,12:,:,:]
WAVD_wm_TmSrs = np.concatenate((WAVD_wm_TmSrs_link1,WAVD_wm_TmSrs_4_5),axis=0)

In [51]:
#stack monthly values so we have two separate time dimensions, one for years, one for months
WAVD_wm_TmSrs_stacked = np.zeros([5,60,12,180,360])

start = 0
finish = 12

for i in range(60):
    for j in range(5):
        WAVD_wm_TmSrs_stacked[j,i,:,:,:] = np.stack(\
        WAVD_wm_TmSrs[j,start:finish,:,:],axis=0)
        
    start+=12
    finish+=12   

In [53]:
WAVD_wm_TmSrs_Annual = np.mean(WAVD_wm_TmSrs_stacked[:,:,:,:,:],axis=2)
WAVD_wm_TmSrs_MAM = np.mean(WAVD_wm_TmSrs_stacked[:,:,2:5,:,:],axis=2)
WAVD_wm_TmSrs_JJA = np.mean(WAVD_wm_TmSrs_stacked[:,:,5:8,:,:],axis=2)
WAVD_wm_TmSrs_SON = np.mean(WAVD_wm_TmSrs_stacked[:,:,8:11,:,:],axis=2)
WAVD_wm_TmSrs_DJF = np.mean((WAVD_wm_TmSrs_stacked[:,:,0,:,:],\
                             WAVD_wm_TmSrs_stacked[:,:,1,:,:],\
                             WAVD_wm_TmSrs_stacked[:,:,11,:,:]),axis=0)

In [54]:
Sims_Iter = range(5)

WAVD_wm_TmSrs_Annual_AA = np.zeros([len(Sims_Iter),Years.size])
WAVD_wm_TmSrs_MAM_AA = np.zeros([len(Sims_Iter),Years.size])
WAVD_wm_TmSrs_JJA_AA = np.zeros([len(Sims_Iter),Years.size])
WAVD_wm_TmSrs_SON_AA = np.zeros([len(Sims_Iter),Years.size])
WAVD_wm_TmSrs_DJF_AA = np.zeros([len(Sims_Iter),Years.size])

WAVD_wm_TmSrs_Annual_GA = np.zeros([len(Sims_Iter),Years.size])
WAVD_wm_TmSrs_MAM_GA = np.zeros([len(Sims_Iter),Years.size])
WAVD_wm_TmSrs_JJA_GA = np.zeros([len(Sims_Iter),Years.size])
WAVD_wm_TmSrs_SON_GA = np.zeros([len(Sims_Iter),Years.size])
WAVD_wm_TmSrs_DJF_GA = np.zeros([len(Sims_Iter),Years.size])

for j in range(len(Sims_Iter)):
    WAVD_wm_TmSrs_Annual_AA[j,:] = Area_Avg.LatLonavg_Time(\
        WAVD_wm_TmSrs_Annual[j,:,150:,:],WACCM_Arctic_Lat,WACCM_Lon)
    
    WAVD_wm_TmSrs_MAM_AA[j,:] = Area_Avg.LatLonavg_Time(\
        WAVD_wm_TmSrs_MAM[j,:,150:,:],WACCM_Arctic_Lat,WACCM_Lon)
    
    WAVD_wm_TmSrs_JJA_AA[j,:] = Area_Avg.LatLonavg_Time(\
        WAVD_wm_TmSrs_JJA[j,:,150:,:],WACCM_Arctic_Lat,WACCM_Lon)
    
    WAVD_wm_TmSrs_SON_AA[j,:] = Area_Avg.LatLonavg_Time(\
        WAVD_wm_TmSrs_SON[j,:,150:,:],WACCM_Arctic_Lat,WACCM_Lon)
    
    WAVD_wm_TmSrs_DJF_AA[j,:] = Area_Avg.LatLonavg_Time(\
        WAVD_wm_TmSrs_DJF[j,:,150:,:],WACCM_Arctic_Lat,WACCM_Lon)
    
    
    WAVD_wm_TmSrs_Annual_GA[j,:] = Area_Avg.LatLonavg_Time(\
        WAVD_wm_TmSrs_Annual[j,:,:,:],WACCM_Lat,WACCM_Lon)
    
    WAVD_wm_TmSrs_MAM_GA[j,:] = Area_Avg.LatLonavg_Time(\
        WAVD_wm_TmSrs_MAM[j,:,:,:],WACCM_Lat,WACCM_Lon)
    
    WAVD_wm_TmSrs_JJA_GA[j,:] = Area_Avg.LatLonavg_Time(\
        WAVD_wm_TmSrs_JJA[j,:,:,:],WACCM_Lat,WACCM_Lon)
    
    WAVD_wm_TmSrs_SON_GA[j,:] = Area_Avg.LatLonavg_Time(\
        WAVD_wm_TmSrs_SON[j,:,:,:],WACCM_Lat,WACCM_Lon)
    
    WAVD_wm_TmSrs_DJF_GA[j,:] = Area_Avg.LatLonavg_Time(\
        WAVD_wm_TmSrs_DJF[j,:,:,:],WACCM_Lat,WACCM_Lon)

In [57]:
WAVD_dOHC_Annual_AA = np.zeros([5])
WAVD_dOHC_MAM_AA = np.zeros([5])
WAVD_dOHC_JJA_AA = np.zeros([5])
WAVD_dOHC_SON_AA = np.zeros([5])
WAVD_dOHC_DJF_AA = np.zeros([5])

WAVD_dOHC_Annual_GA = np.zeros([5])
WAVD_dOHC_MAM_GA = np.zeros([5])
WAVD_dOHC_JJA_GA = np.zeros([5])
WAVD_dOHC_SON_GA = np.zeros([5])
WAVD_dOHC_DJF_GA = np.zeros([5])

for i in range(5):
    WAVD_dOHC_Annual_AA[i] = (np.mean(WAVD_wm_TmSrs_Annual_AA[i,50:]))-\
                                     (np.mean(WAVD_wm_TmSrs_Annual_AA[i,:10]))
    WAVD_dOHC_MAM_AA[i] = (np.mean(WAVD_wm_TmSrs_MAM_AA[i,50:]))-\
                                     (np.mean(WAVD_wm_TmSrs_MAM_AA[i,:10]))
    WAVD_dOHC_JJA_AA[i] = (np.mean(WAVD_wm_TmSrs_JJA_AA[i,50:]))-\
                                     (np.mean(WAVD_wm_TmSrs_JJA_AA[i,:10]))
    WAVD_dOHC_SON_AA[i] = (np.mean(WAVD_wm_TmSrs_SON_AA[i,50:]))-\
                                     (np.mean(WAVD_wm_TmSrs_SON_AA[i,:10]))
    WAVD_dOHC_DJF_AA[i] = (np.mean(WAVD_wm_TmSrs_DJF_AA[i,50:]))-\
                                     (np.mean(WAVD_wm_TmSrs_DJF_AA[i,:10]))

    WAVD_dOHC_Annual_GA[i] = (np.mean(WAVD_wm_TmSrs_Annual_GA[i,50:]))-\
                                     (np.mean(WAVD_wm_TmSrs_Annual_GA[i,:10]))
    WAVD_dOHC_MAM_GA[i] = (np.mean(WAVD_wm_TmSrs_MAM_GA[i,50:]))-\
                                     (np.mean(WAVD_wm_TmSrs_MAM_GA[i,:10]))
    WAVD_dOHC_JJA_GA[i] = (np.mean(WAVD_wm_TmSrs_JJA_GA[i,50:]))-\
                                     (np.mean(WAVD_wm_TmSrs_JJA_GA[i,:10]))
    WAVD_dOHC_SON_GA[i] = (np.mean(WAVD_wm_TmSrs_SON_GA[i,50:]))-\
                                     (np.mean(WAVD_wm_TmSrs_SON_GA[i,:10]))
    WAVD_dOHC_DJF_GA[i] = (np.mean(WAVD_wm_TmSrs_DJF_GA[i,50:]))-\
                                     (np.mean(WAVD_wm_TmSrs_DJF_GA[i,:10]))

In [61]:
print('Annual :',WAVD_dOHC_Annual_AA)
print('\n')
print('MAM :',WAVD_dOHC_MAM_AA)
print('JJA :',WAVD_dOHC_JJA_AA)
print('SON :',WAVD_dOHC_SON_AA)
print('DJF :',WAVD_dOHC_DJF_AA)

Annual : [0.76140808 0.59305529 1.11984282 0.76136139 1.05061791]


MAM : [0.51699401 1.79960317 2.98954838 1.38505778 1.97627787]
JJA : [7.20964061 5.35060163 6.71787862 5.69165004 6.69939746]
SON : [-1.58318139 -2.61027099 -2.4264485  -2.17455838 -1.82597202]
DJF : [-3.09782092 -2.16771263 -2.80160722 -1.85670389 -2.64723169]


In [63]:
print('Annual :',WAVD_dOHC_Annual_GA)
print('\n')
print('MAM :',WAVD_dOHC_MAM_GA)
print('JJA :',WAVD_dOHC_JJA_GA)
print('SON :',WAVD_dOHC_SON_GA)
print('DJF :',WAVD_dOHC_DJF_GA)

Annual : [0.79452221 0.99107548 0.98978734 0.81706571 0.86675488]


MAM : [0.16826313 0.5120441  0.3955495  0.50087851 0.32662805]
JJA : [0.88512355 1.04117227 1.15787573 0.6025709  0.83412033]
SON : [1.27493161 1.08406636 0.92434752 1.08183564 1.36940598]
DJF : [0.84977054 1.3270192  1.48137661 1.08297779 0.93686516]


In [65]:
WAVD_OCN_EnergyB_Annual_AA_file = open('WAVD_OCN_EnergyB_AA.pickle','wb')
pickle.dump(WAVD_dOHC_Annual_AA,WAVD_OCN_EnergyB_Annual_AA_file)
WAVD_OCN_EnergyB_Annual_AA_file.close()

#seasonal
WAVD_OCN_EnergyB_MAM_AA_file = open('WAVD_OCN_EnergyB_MAM_AA.pickle','wb')
pickle.dump(WAVD_dOHC_MAM_AA,WAVD_OCN_EnergyB_MAM_AA_file)
WAVD_OCN_EnergyB_MAM_AA_file.close()

WAVD_OCN_EnergyB_JJA_AA_file = open('WAVD_OCN_EnergyB_JJA_AA.pickle','wb')
pickle.dump(WAVD_dOHC_JJA_AA,WAVD_OCN_EnergyB_JJA_AA_file)
WAVD_OCN_EnergyB_JJA_AA_file.close()

WAVD_OCN_EnergyB_SON_AA_file = open('WAVD_OCN_EnergyB_SON_AA.pickle','wb')
pickle.dump(WAVD_dOHC_SON_AA,WAVD_OCN_EnergyB_SON_AA_file)
WAVD_OCN_EnergyB_SON_AA_file.close()

WAVD_OCN_EnergyB_DJF_AA_file = open('WAVD_OCN_EnergyB_DJF_AA.pickle','wb')
pickle.dump(WAVD_dOHC_DJF_AA,WAVD_OCN_EnergyB_DJF_AA_file)
WAVD_OCN_EnergyB_DJF_AA_file.close()

WAVD_OCN_EnergyB_Annual_GA_file = open('WAVD_OCN_EnergyB_GA.pickle','wb')
pickle.dump(WAVD_dOHC_Annual_GA,WAVD_OCN_EnergyB_Annual_GA_file)
WAVD_OCN_EnergyB_Annual_GA_file.close()

#seasonal
WAVD_OCN_EnergyB_MAM_GA_file = open('WAVD_OCN_EnergyB_MAM_GA.pickle','wb')
pickle.dump(WAVD_dOHC_MAM_GA,WAVD_OCN_EnergyB_MAM_GA_file)
WAVD_OCN_EnergyB_MAM_GA_file.close()

WAVD_OCN_EnergyB_JJA_GA_file = open('WAVD_OCN_EnergyB_JJA_GA.pickle','wb')
pickle.dump(WAVD_dOHC_JJA_GA,WAVD_OCN_EnergyB_JJA_GA_file)
WAVD_OCN_EnergyB_JJA_GA_file.close()

WAVD_OCN_EnergyB_SON_GA_file = open('WAVD_OCN_EnergyB_SON_GA.pickle','wb')
pickle.dump(WAVD_dOHC_SON_GA,WAVD_OCN_EnergyB_SON_GA_file)
WAVD_OCN_EnergyB_SON_GA_file.close()

WAVD_OCN_EnergyB_DJF_GA_file = open('WAVD_OCN_EnergyB_DJF_GA.pickle','wb')
pickle.dump(WAVD_dOHC_DJF_GA,WAVD_OCN_EnergyB_DJF_GA_file)
WAVD_OCN_EnergyB_DJF_GA_file.close()

In [66]:
WAVD_dST_CAM3Grid = np.swapaxes(pickle.load(open(\
   "Future Projection CC Responses/wAVD_dST_CAM3Grid.pickle","rb"),encoding='latin1'),2,3)

WAVD_dST_CAM3Grid_AA = np.zeros([5,12])
WAVD_dST_CAM3Grid_GA = np.zeros([5,12])

for i in range(5):
    WAVD_dST_CAM3Grid_AA[i,:] = Area_Avg.LatLonavg_Time(\
    WAVD_dST_CAM3Grid[i,:,54:,:],CAM3_Lat[54:],CAM3_Lon)
    
    WAVD_dST_CAM3Grid_GA[i,:] = Area_Avg.LatLonavg_Time(\
    WAVD_dST_CAM3Grid[i,:,:,:],CAM3_Lat,CAM3_Lon)

WAVD_dST_Annual_CAM3Grid_AA = np.mean(WAVD_dST_CAM3Grid_AA,axis=1)
WAVD_dST_SON_CAM3Grid_AA = np.mean(WAVD_dST_CAM3Grid_AA[:,8:11],axis=1)
WAVD_dST_JJA_CAM3Grid_AA = np.mean(WAVD_dST_CAM3Grid_AA[:,5:8],axis=1)
WAVD_dST_MAM_CAM3Grid_AA = np.mean(WAVD_dST_CAM3Grid_AA[:,2:5],axis=1)
WAVD_dST_DJF_CAM3Grid_AA = np.mean((WAVD_dST_CAM3Grid_AA[:,0],\
                                    WAVD_dST_CAM3Grid_AA[:,1],\
                                    WAVD_dST_CAM3Grid_AA[:,11]),axis=0)

WAVD_dST_Annual_CAM3Grid_GA = np.mean(WAVD_dST_CAM3Grid_GA,axis=1)
WAVD_dST_SON_CAM3Grid_GA = np.mean(WAVD_dST_CAM3Grid_GA[:,8:11],axis=1)
WAVD_dST_JJA_CAM3Grid_GA = np.mean(WAVD_dST_CAM3Grid_GA[:,5:8],axis=1)
WAVD_dST_MAM_CAM3Grid_GA = np.mean(WAVD_dST_CAM3Grid_GA[:,2:5],axis=1)
WAVD_dST_DJF_CAM3Grid_GA = np.mean((WAVD_dST_CAM3Grid_GA[:,0],\
                                    WAVD_dST_CAM3Grid_GA[:,1],\
                                    WAVD_dST_CAM3Grid_GA[:,11]),axis=0)

In [67]:
WAVD_dOHC_Feedback_Annual_AA = WAVD_dOHC_Annual_AA/WAVD_dST_Annual_CAM3Grid_AA
WAVD_dOHC_Feedback_SON_AA = WAVD_dOHC_SON_AA/WAVD_dST_SON_CAM3Grid_AA
WAVD_dOHC_Feedback_JJA_AA = WAVD_dOHC_JJA_AA/WAVD_dST_JJA_CAM3Grid_AA
WAVD_dOHC_Feedback_MAM_AA = WAVD_dOHC_MAM_AA/WAVD_dST_MAM_CAM3Grid_AA
WAVD_dOHC_Feedback_DJF_AA = WAVD_dOHC_DJF_AA/WAVD_dST_DJF_CAM3Grid_AA

WAVD_dOHC_Feedback_Annual_GA = WAVD_dOHC_Annual_GA/WAVD_dST_Annual_CAM3Grid_GA
WAVD_dOHC_Feedback_SON_GA = WAVD_dOHC_SON_GA/WAVD_dST_SON_CAM3Grid_GA
WAVD_dOHC_Feedback_JJA_GA = WAVD_dOHC_JJA_GA/WAVD_dST_JJA_CAM3Grid_GA
WAVD_dOHC_Feedback_MAM_GA = WAVD_dOHC_MAM_GA/WAVD_dST_MAM_CAM3Grid_GA
WAVD_dOHC_Feedback_DJF_GA = WAVD_dOHC_DJF_GA/WAVD_dST_DJF_CAM3Grid_GA

In [68]:
WAVD_OCN_Feedback_Annual_AA_file = open('WAVD_OCN_Feedback_AA.pickle','wb')
pickle.dump(WAVD_dOHC_Feedback_Annual_AA,WAVD_OCN_Feedback_Annual_AA_file)
WAVD_OCN_Feedback_Annual_AA_file.close()

#seasonal
WAVD_OCN_Feedback_MAM_AA_file = open('WAVD_OCN_Feedback_MAM_AA.pickle','wb')
pickle.dump(WAVD_dOHC_Feedback_MAM_AA,WAVD_OCN_Feedback_MAM_AA_file)
WAVD_OCN_Feedback_MAM_AA_file.close()

WAVD_OCN_Feedback_JJA_AA_file = open('WAVD_OCN_Feedback_JJA_AA.pickle','wb')
pickle.dump(WAVD_dOHC_Feedback_JJA_AA,WAVD_OCN_Feedback_JJA_AA_file)
WAVD_OCN_Feedback_JJA_AA_file.close()

WAVD_OCN_Feedback_SON_AA_file = open('WAVD_OCN_Feedback_SON_AA.pickle','wb')
pickle.dump(WAVD_dOHC_Feedback_SON_AA,WAVD_OCN_Feedback_SON_AA_file)
WAVD_OCN_Feedback_SON_AA_file.close()

WAVD_OCN_Feedback_DJF_AA_file = open('WAVD_OCN_Feedback_DJF_AA.pickle','wb')
pickle.dump(WAVD_dOHC_Feedback_DJF_AA,WAVD_OCN_Feedback_DJF_AA_file)
WAVD_OCN_Feedback_DJF_AA_file.close()

WAVD_OCN_Feedback_Annual_GA_file = open('WAVD_OCN_Feedback_GA.pickle','wb')
pickle.dump(WAVD_dOHC_Feedback_Annual_GA,WAVD_OCN_Feedback_Annual_GA_file)
WAVD_OCN_Feedback_Annual_GA_file.close()

#seasonal
WAVD_OCN_Feedback_MAM_GA_file = open('WAVD_OCN_Feedback_MAM_GA.pickle','wb')
pickle.dump(WAVD_dOHC_Feedback_MAM_GA,WAVD_OCN_Feedback_MAM_GA_file)
WAVD_OCN_Feedback_MAM_GA_file.close()

WAVD_OCN_Feedback_JJA_GA_file = open('WAVD_OCN_Feedback_JJA_GA.pickle','wb')
pickle.dump(WAVD_dOHC_Feedback_JJA_GA,WAVD_OCN_Feedback_JJA_GA_file)
WAVD_OCN_Feedback_JJA_GA_file.close()

WAVD_OCN_Feedback_SON_GA_file = open('WAVD_OCN_Feedback_SON_GA.pickle','wb')
pickle.dump(WAVD_dOHC_Feedback_SON_GA,WAVD_OCN_Feedback_SON_GA_file)
WAVD_OCN_Feedback_SON_GA_file.close()

WAVD_OCN_Feedback_DJF_GA_file = open('WAVD_OCN_Feedback_DJF_GA.pickle','wb')
pickle.dump(WAVD_dOHC_Feedback_DJF_GA,WAVD_OCN_Feedback_DJF_GA_file)
WAVD_OCN_Feedback_DJF_GA_file.close()